In [ ]:
using OceananigansLagrangianFilter: create_filtered_vars, create_forcing, create_output_fields, initialise_filtered_vars_from_model, zero_closure_for_filtered_vars
using OceananigansLagrangianFilter: regrid_to_mean_position!, compute_Eulerian_filter!, compute_time_shift!, jld2_to_netcdf
using OceananigansLagrangianFilter
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CairoMakie 
using NCDatasets
using Printf

Precompiling packages...
   3565.2 ms  ✓ OceananigansLagrangianFilter
  1 dependency successfully precompiled in 5 seconds. 356 already precompiled.


In [2]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.1             # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front

filename_stem = "periodic_geostrophic_adjustment_online"

"periodic_geostrophic_adjustment_online"

In [3]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [4]:

# Define tracers
tracers = (:b,:T)

# Define forcing
forcing = NamedTuple()


NamedTuple()

In [5]:
# Define filter configuration

filter_params = (;a1 = f/4, c1 = f/4, N_coeffs = 0.5)
filter_config = OnlineFilterConfig( grid = grid,
                                    output_filename = filename_stem * ".jld2",
                                    var_names_to_filter = ("b","T"), 
                                    velocity_names = ("u","w"),
                                    filter_params = filter_params)

OnlineFilterConfig(300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25, "periodic_geostrophic_adjustment_online.jld2", ("b", "T"), ("u", "w"), (a1 = 2.5e-5, c1 = 2.5e-5, N_coeffs = 0.5), true, 5, "online")

In [6]:
# Now we define the filtering tracers and forcings
# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(filter_config)
tracers = (filtered_vars..., tracers...)

# Create forcing for these filtered variables
filter_forcing = create_forcing(filtered_vars, filter_config)

forcing = merge(forcing, filter_forcing);

In [7]:
# Define closure for model and filtered variables
# Helper to set filtered variable closures to zero, otherwise they get set to the same as other tracers
zero_filtered_closure = zero_closure_for_filtered_vars(filter_config)

horizontal_closure = HorizontalScalarDiffusivity(ν=1e-6, κ=merge((T=1e-6, b= 1e-6),zero_filtered_closure) )
vertical_closure = VerticalScalarDiffusivity(ν=1e-6 , κ=merge((T=1e-6, b= 1e-6),zero_filtered_closure) )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-6, κ=(T=1.0e-6, b=1.0e-6, T_C1=0.0, xi_u_C1=0.0, xi_w_C1=0.0, b_C1=0.0)), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-6, κ=(T=1.0e-6, b=1.0e-6, T_C1=0.0, xi_u_C1=0.0, xi_w_C1=0.0, b_C1=0.0)))

In [8]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = tracers,
                forcing = forcing,
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b_C1, T_C1, xi_u_C1, xi_w_C1, b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-6, κ=(b_C1=0.0, T_C1=0.0, xi_u_C1=0.0, xi_w_C1=0.0, b=1.0e-6, T=1.0e-6))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-6, κ=(b_C1=0.0, T_C1=0.0, xi_u_C1=0.0, xi_w_C1=0.0, b=1.0e-6, T=1.0e-6))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [9]:
# Set initial conditions of actual variables
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/20)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [10]:
# Set appropriate initial conditions for the filtered variables based on the actual variables
initialise_filtered_vars_from_model(model, filter_config)


In [11]:
simulation = Simulation(model, Δt=1hour, stop_time=3days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 1 hour
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 3 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [12]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [13]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, (max(u), max(b_C1), max(T_C1)) : (%6.3e, %6.3e, %6.3e), next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, model.tracers.b_C1), maximum(abs, model.tracers.T_C1), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

In [14]:
# Output setup
# Create filtered outputs
outputs = create_output_fields(model, filter_config)

# Add in original variables if desired
outputs["b"] = model.tracers.b
outputs["T"] = model.tracers.T
outputs["u"] = model.velocities.u
outputs["v"] = model.velocities.v
outputs["w"] = model.velocities.w



simulation.output_writers[:jld2fields] = JLD2Writer(
    model, outputs, filename=filename_stem * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


rm(filename_stem * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, outputs, filename=filename_stem * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)

    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment_online.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 9 outputs: (v, xi_u, w, T, b, T_Lagrangian_filtered, xi_w, u, b_Lagrangian_filtered)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 28.9 KiB

In [15]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 42.228 seconds, (max(u), max(b_C1), max(T_C1)) : (0.000e+00, 4.000e+00, 3.996e+04), next Δt: 20 minutes


[ Info:     ... simulation initialization complete (21.647 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (4.145 seconds).


[20.63%] i: 100, t: 14.857 hours, wall time: 8.125 seconds, (max(u), max(b_C1), max(T_C1)) : (2.560e-02, 4.000e+00, 4.000e+04), next Δt: 4.039 minutes
[30.65%] i: 200, t: 22.065 hours, wall time: 533.123 ms, (max(u), max(b_C1), max(T_C1)) : (3.061e-02, 4.000e+00, 4.000e+04), next Δt: 3.629 minutes
[40.52%] i: 300, t: 1.216 days, wall time: 540.638 ms, (max(u), max(b_C1), max(T_C1)) : (2.642e-02, 4.000e+00, 4.000e+04), next Δt: 4.204 minutes
[49.69%] i: 400, t: 1.491 days, wall time: 551.214 ms, (max(u), max(b_C1), max(T_C1)) : (1.040e-02, 4.000e+00, 4.000e+04), next Δt: 5.146 minutes
[59.29%] i: 500, t: 1.779 days, wall time: 555.282 ms, (max(u), max(b_C1), max(T_C1)) : (1.347e-02, 4.000e+00, 4.000e+04), next Δt: 4.548 minutes
[69.35%] i: 600, t: 2.080 days, wall time: 567.603 ms, (max(u), max(b_C1), max(T_C1)) : (2.536e-02, 4.000e+00, 4.000e+04), next Δt: 4.085 minutes
[79.44%] i: 700, t: 2.383 days, wall time: 589.552 ms, (max(u), max(b_C1), max(T_C1)) : (3.016e-02, 4.000e+00, 4.000e

[ Info: Simulation is stopping after running for 31.405 seconds.
[ Info: Simulation time 3 days equals or exceeds stop time 3 days.
[ Info: Simulation completed in 31.433 seconds


In [16]:
# Regrid if desired
if filter_config.map_to_mean
    regrid_to_mean_position!(filter_config)
end


[ Info: Assuming velocities normal to z boundaries are zero
[ Info: Wrote regridded data to new variables with _at_mean suffix in file periodic_geostrophic_adjustment_online.jld2


In [17]:
# Calculate Eulerian filter if desired
compute_Eulerian_filter!(filter_config)

[ Info: Computing Eulerian filter for variable b
[ Info: Computing Eulerian filter at time 0.0 =  (index 1 of 73)
[ Info: Computing Eulerian filter at time 3600.0 =  (index 2 of 73)
[ Info: Computing Eulerian filter at time 7200.0 =  (index 3 of 73)
[ Info: Computing Eulerian filter at time 10800.0 =  (index 4 of 73)
[ Info: Computing Eulerian filter at time 14400.0 =  (index 5 of 73)
[ Info: Computing Eulerian filter at time 18000.0 =  (index 6 of 73)
[ Info: Computing Eulerian filter at time 21600.0 =  (index 7 of 73)
[ Info: Computing Eulerian filter at time 25200.0 =  (index 8 of 73)
[ Info: Computing Eulerian filter at time 28800.0 =  (index 9 of 73)
[ Info: Computing Eulerian filter at time 32400.0 =  (index 10 of 73)
[ Info: Computing Eulerian filter at time 36000.0 =  (index 11 of 73)
[ Info: Computing Eulerian filter at time 39600.0 =  (index 12 of 73)
[ Info: Computing Eulerian filter at time 43200.0 =  (index 13 of 73)
[ Info: Computing Eulerian filter at time 46800.0 =  (in

In [ ]:

# Add a shifted time coordinate if desired
compute_time_shift!(filter_config)


# Output netcdf if desired
jld2_to_netcdf(filename_stem * ".jld2", filename_stem * ".nc")


[ Info: Wrote time shift data to new group timeseries/t_shifted in file periodic_geostrophic_adjustment_online.jld2


LoadError: UndefVarError: `filename` not defined